<a href="https://colab.research.google.com/github/LeonardoVieiraGuimaraes/MiniCursoPalestra/blob/main/redeNeuralArtificial/ObjetoQueda/CorpoQueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1 — preparação do ambiente e fórmula analítica (exemplo)
import numpy as np
import pandas as pd
import os
# detectar Colab
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    print('Executando no Colab — se desejar salvar modelos monte o Drive: from google.colab import drive; drive.mount(...)')

# Parâmetros de exemplo para demonstração
s_0 = 0.0
m = 5.0  # kg
k = 0.5  # coeficiente de arrasto (modelo linear usado aqui)
t = 5.0  # segundos
g = 9.81  # aceleração da gravidade (m/s^2)

# Fórmulas analíticas (modelo com arrasto linear)
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

print(f'Velocidade analítica: {v_analitica:.4f} m/s')
print(f'Posição analítica: {y_analitica:.4f} m')

Executando no Colab — se desejar salvar modelos monte o Drive: from google.colab import drive; drive.mount(...)
Velocidade analítica: 38.5993 m/s
Posição analítica: 104.5066 m


In [2]:
# Célula 2 — gerar dataset de exemplos (modelo analítico com arrasto linear)
# Definindo constantes
g = 9.81  # aceleração da gravidade (m/s^2)

# Geração de dados
np.random.seed(42)
n_samples = 1000

massas = np.random.uniform(1.0, 10.0, n_samples)  # massa entre 1 e 10 kg
k_arrastos = np.random.uniform(0.1, 1.0, n_samples)  # constante de arrasto entre 0.1 e 1
tempos = np.random.uniform(0.0, 10.0, n_samples)  # tempo entre 0 e 10 segundos

velocidades = []
posicoes = []

for m_val, k_val, t_val in zip(massas, k_arrastos, tempos):
    v_t = (m_val * g / k_val) * (1 - np.exp(-k_val * t_val / m_val))
    y_t = (m_val * g / k_val) * t_val - (m_val**2 * g / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
    velocidades.append(v_t)
    posicoes.append(y_t)

# Criação do DataFrame
df = pd.DataFrame({
    'massa': massas,
    'constante_de_arrasto': k_arrastos,
    'tempo': tempos,
    'velocidade': velocidades,
    'posicao': posicoes
})
# Salvar em um arquivo CSV local
os.makedirs('data', exist_ok=True)
csv_path = 'data/queda_objeto.csv'
df.to_csv(csv_path, index=False)
print('Dataset salvo em', csv_path)
df.head()

Dataset salvo em data/queda_objeto.csv


,massa,constante_de_arrasto,tempo,velocidade,posicao
0,4.370861,0.266620,2.617057,23.728928,31.875755
1,9.556429,0.587711,2.469788,22.478312,28.460762
2,7.587945,0.885651,9.062546,54.864287,291.636528
3,6.387926,0.759002,2.495462,21.184788,27.737263
4,2.404168,0.825905,2.719497,17.336937,27.192141


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Carregar os dados do CSV (caminho atualizado)
csv_path = 'data/queda_objeto.csv'
if not os.path.exists(csv_path):
    raise SystemExit(f'Arquivo de dados não encontrado: {csv_path} - execute a célula de geração de dados primeiro')
df = pd.read_csv(csv_path)

# Separar as entradas (X) e as saídas (y)
X = df[['massa', 'constante_de_arrasto', 'tempo']]
y = df[['velocidade', 'posicao']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# salvar scalers para inferência posterior
os.makedirs('model', exist_ok=True)
import joblib
joblib.dump(scaler_X, 'model/scaler_X.pkl')
joblib.dump(scaler_y, 'model/scaler_y.pkl')
print('Scalers salvos em model/')

Scalers salvos em model/


In [4]:
# Construir a rede neural
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Saída de 2 dimensões: velocidade e posição
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,546 (17.76 KB)

 Trainable params: 4,546 (17.76 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Treinar o modelo com callbacks (salvar melhor modelo)
checkpoint_cb = ModelCheckpoint('model/queda_objeto_best.keras', save_best_only=True, monitor='val_loss')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[checkpoint_cb, earlystop_cb])

# salvar o modelo final
model.save('model/queda_objeto.keras')
print('Modelo salvo em model/queda_objeto.keras')

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.9597 - mae: 0.8114 - val_loss: 0.4341 - val_mae: 0.5491
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3343 - mae: 0.4682 - val_loss: 0.0883 - val_mae: 0.2208
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0602 - mae: 0.1852 - val_loss: 0.0419 - val_mae: 0.1498
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0265 - mae: 0.1275 - val_loss: 0.0272 - val_mae: 0.1142
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0185 - mae: 0.1029 - val_loss: 0.0219 - val_mae: 0.1055
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0149 - mae: 0.0928 - val_loss: 0.0199 - val_mae: 0.0960
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0130 - mae: 0.0844 - val_loss: 0.0171 - val_mae: 0.0897
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0124 - mae: 0.0816 - val_loss: 0.0157 - val_mae: 0.0870
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.01

In [6]:
# Avaliar o modelo (carregar melhor modelo salvo se existir)
import os
best_path = 'model/queda_objeto_best.keras'
if os.path.exists(best_path):
    from tensorflow.keras.models import load_model
    model = load_model(best_path)
    print('Carregado melhor modelo salvo em', best_path)

loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae:.6f}')

# Modelo final já salvo na etapa de treino como model/queda_objeto.keras

Carregado melhor modelo salvo em model/queda_objeto_best.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.1280e-04 - mae: 0.0166  
Mean Absolute Error on test data: 0.016512


In [7]:
# Célula de inferência — carregar modelo e scalers e testar uma entrada
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib

model_path = 'model/queda_objeto.keras'
if not os.path.exists(model_path):
    raise SystemExit('Modelo não encontrado: ' + model_path + ' - execute a célula de treino primeiro')
model = load_model(model_path)
scaler_X = joblib.load('model/scaler_X.pkl')
scaler_y = joblib.load('model/scaler_y.pkl')

# Exemplo de entrada e previsão
m_val = 5.0
k_val = 0.5
t_val = 5.0
entrada = np.array([[m_val, k_val, t_val]])
entrada_pad = scaler_X.transform(entrada)
saida_pad = model.predict(entrada_pad)
saida = scaler_y.inverse_transform(saida_pad)
v_pred, y_pred = saida[0]
print(f'Previsão da rede — velocidade: {v_pred:.4f} m/s, posição: {y_pred:.4f} m')

# comparar com solução analítica (mesmo modelo linear usado para gerar dados)
v_analitica = (m_val * 9.81 / k_val) * (1 - np.exp(-k_val * t_val / m_val))
y_analitica = (m_val * 9.81 / k_val) * t_val - (m_val**2 * 9.81 / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
print(f'Analítica — velocidade: {v_analitica:.4f} m/s, posição: {y_analitica:.4f} m')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Previsão da rede — velocidade: 38.5483 m/s, posição: 104.2930 m
Analítica — velocidade: 38.5993 m/s, posição: 104.5066 m


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
